![walmartecomm](walmartecomm.jpg)

Walmart is the biggest retail store in the United States. Just like others, they have been expanding their e-commerce part of the business. By the end of 2022, e-commerce represented a roaring $80 billion in sales, which is 13% of total sales of Walmart. One of the main factors that affects their sales is public holidays, like the Super Bowl, Labour Day, Thanksgiving, and Christmas. 

In this project, you have been tasked with creating a data pipeline for the analysis of supply and demand around the holidays, along with conducting a preliminary analysis of the data. You will be working with two data sources: grocery sales and complementary data. You have been provided with the `grocery_sales` table in `PostgreSQL` database with the following features:

# `grocery_sales`
- `"index"` - unique ID of the row
- `"Store_ID"` - the store number
- `"Date"` - the week of sales
- `"Weekly_Sales"` - sales for the given store

Also, you have the `extra_data.parquet` file that contains complementary data:

# `extra_data.parquet`
- `"IsHoliday"` - Whether the week contains a public holiday - 1 if yes, 0 if no.
- `"Temperature"` - Temperature on the day of sale
- `"Fuel_Price"` - Cost of fuel in the region
- `"CPI"` – Prevailing consumer price index
- `"Unemployment"` - The prevailing unemployment rate
- `"MarkDown1"`, `"MarkDown2"`, `"MarkDown3"`, `"MarkDown4"` - number of promotional markdowns
- `"Dept"` - Department Number in each store
- `"Size"` - size of the store
- `"Type"` - type of the store (depends on `Size` column)

You will need to merge those files and perform some data manipulations. The transformed DataFrame can then be stored as the `clean_data` variable containing the following columns:
- `"Store_ID"`
- `"Month"`
- `"Dept"`
- `"IsHoliday"`
- `"Weekly_Sales"`
- `"CPI"`
- "`"Unemployment"`"

After merging and cleaning the data, you will have to analyze monthly sales of Walmart and store the results of your analysis as the `agg_data` variable that should look like:

|  Month | Weekly_Sales  | 
|---|---|
| 1.0  |  33174.178494 |
|  2.0 |  34333.326579 |
|  ... | ...  |  

Finally, you should save the `clean_data` and `agg_data` as the csv files.

It is recommended to use `pandas` for this project. 

In [230]:
import pandas as pd
import os

# Extract function is already implemented for you 
def extract(store_data, extra_data):
    extra_df = pd.read_parquet(extra_data)
    merged_df = store_data.merge(extra_df, on = "index")
    return merged_df

# Call the extract() function and store it as the "merged_df" variable
merged_df = extract(grocery_sales, "extra_data.parquet")

In [231]:
merged_df.head()
print(merged_df.isna().sum())

index            0
Store_ID         0
Date            39
Dept             0
Weekly_Sales    38
IsHoliday        0
Temperature      0
Fuel_Price       0
MarkDown1        0
MarkDown2        0
MarkDown3        0
MarkDown4        1
MarkDown5        1
CPI             47
Unemployment    37
Type             1
Size             1
dtype: int64


In [232]:
# Create the transform() function with one parameter: "raw_data"
def transform(raw_data):
    # Write your code here
    raw_data = raw_data[raw_data["Weekly_Sales"] > 10000]
    
    raw_data["Weekly_Sales"] = raw_data["Weekly_Sales"].fillna(0)
    raw_data["CPI"] = raw_data["CPI"].interpolate()
    raw_data["Unemployment"] = raw_data["Unemployment"].ffill()
    
    raw_data["Date"] = pd.to_datetime(raw_data["Date"])
    raw_data["Month"] = raw_data["Date"].dt.month
    raw_data = raw_data.dropna(subset=["Date", "MarkDown4", "MarkDown5"])
    raw_data = raw_data.drop(
        columns=["index", "Temperature", "Fuel_Price", "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5","Type","Size","Dept","Date"]
    )
 
    return raw_data

In [233]:
# Call the transform() function and pass the merged DataFrame
clean_data = transform(merged_df)
clean_data.head()

,Store_ID,Weekly_Sales,IsHoliday,CPI,Unemployment,Month
0,1,24924.50,0,211.096358,8.106,2.0
1,1,11737.12,0,211.096358,8.106,2.0
2,1,13223.76,0,211.096358,8.106,2.0
5,1,46729.77,0,211.096358,8.106,2.0
6,1,21249.31,0,211.096358,8.106,2.0


In [234]:
def avg_weekly_sales_per_month(clean_data):
    agg_data = (clean_data[["Month", "Weekly_Sales"]]  # Select necessary columns
                .groupby("Month")  # Group by Month
                .agg(Avg_Sales=("Weekly_Sales", "mean")) # Calculate average weekly sales
                .reset_index()  # Reset index
                .round(2)  # Round to two decimal places
               )
    return agg_data

In [235]:
# Call the avg_weekly_sales_per_month() function and pass the cleaned DataFrame
agg_data=avg_weekly_sales_per_month(clean_data)
agg_data

,Month,Avg_Sales
0,1.0,33174.18
1,2.0,34342.44
2,3.0,33227.31
3,4.0,33414.78
4,5.0,33339.89
5,6.0,34582.47
6,7.0,33930.77
7,8.0,33644.79
8,9.0,33266.59
9,10.0,32736.99


In [236]:
# Create the load() function that takes in the cleaned DataFrame and the aggregated one with the paths where they are going to be stored
def load(full_data, full_data_file_path, agg_data, agg_data_file_path):
    # Write your code here
    full_data.to_csv(full_data_file_path,index=False)
    agg_data.to_csv(agg_data_file_path,index=False)

In [237]:
# Call the load() function and pass the cleaned and aggregated DataFrames with their paths
load(clean_data,"clean_data.csv",agg_data,"agg_data.csv")

In [238]:
# Create the validation() function with one parameter: file_path - to check whether the previous function was correctly executed
def validation(file_path):
    # Write your code here
    return os.path.exists(file_path)

In [239]:
# Call the validation() function and pass first, the cleaned DataFrame path, and then the aggregated DataFrame path
print(validation("clean_data.csv"))
print(validation("agg_data.csv"))

True
True
